# Tracking NEOCP for Observatory Follow-Up

This example shows several ways you can use adam-core to generate ephemeris for objects on the NEOCP. First you will need a couple packages:

```bash
pip install adam-core
pip install adam-assist
```


In [ ]:
# First let's fetch the objects in question
from adam_core.orbits.query.scout import query_scout, get_scout_objects

# Using the CNEOS Scout API, we can conveniently get a list of objects currently in 
# the NEOCP
scout_objects = get_scout_objects()

# You can preview the objects like so:
print(scout_objects.to_dataframe())

In [ ]:
# Decide which objects you want to track. For now, we select the first one.
object_of_interest = scout_objects[10]

# You could also select by object name like so:
# object_of_interest = scout_objects.select("objectName", "P126Tdm")
# Or you could sort by something like phaScore like so:
# sorted_by_pha_score = scout_objects.sort_by([("phaScore", "descending")])

# With your objects chosen, we send in an array of ids / object names to the Scout API
# This gives us the Scout variants for each object
samples = query_scout(object_of_interest.objectName)

# You generally get back 1000 variants from Scout
# VariantOrbits(size=1000)

# Let's collapse the sampled orbits into an uncertainty.
orbits = samples.collapse_by_object_id()

In [4]:
from adam_core.time import Timestamp
from adam_core.observers import Observers

# Decide on the exposure times you want to use
# Lots of options here, including from astropy.time.Time objects, from MJD, JD, etc.
times = Timestamp.from_iso8601(
    [
        "2025-02-23T00:00:00Z", "2025-02-23T00:05:00Z", "2025-02-23T00:10:00Z"
    ], scale="utc"
)

# Now we define observer positions from an observatory code
observers = Observers.from_code("T08", times)


In [ ]:
# Now we can generate some on-sky ephemeris with uncertainty
# You could also use the Scout API directly to do this: https://ssd-api.jpl.nasa.gov/doc/scout.html
from adam_assist import ASSISTPropagator
propagator = ASSISTPropagator()

ephemeris = propagator.generate_ephemeris(
    orbits,
    observers,
    covariance=True,
    num_samples=1000,
    max_processes=10  # Using multiprocessing helps speed things up
)

In [ ]:
# Now we can iterate through the unique times and determine where to point
for ephem in ephemeris:
    print(f"Object ID: {ephem.object_id[0]}")
    print(f"\nTime: {ephem.coordinates.time.to_iso8601()[0]}")
    print(f"RA: {ephem.coordinates.lon[0]}, Sigma RA: {ephem.coordinates.sigma_lon[0]}")
    print(f"DEC: {ephem.coordinates.lat[0]}, Sigma DEC: {ephem.coordinates.sigma_lat[0]}")


In [ ]:
# The Ephemeris object is a Quivr Table (based on pyarrow), but you can also convert to a pandas DataFrame
ephemeris.to_dataframe()
# That's it, you're all set!


In [ ]:
# Alternatively, if you want you can propagate the Scout samples directly
# and use their distribution to ascertain uncertainty in the on-sky location
import pyarrow.compute as pc

sample_direct_ephem = propagator.generate_ephemeris(
    samples,
    observers,
    max_processes=10
)

unique_times = sample_direct_ephem.coordinates.time.unique()
for unique_time in unique_times:
    sample_ephem_time = sample_direct_ephem.apply_mask(sample_direct_ephem.coordinates.time.equals(unique_time))
    print(f"\nObject ID: {sample_ephem_time.object_id[0]}")
    print(f"Time: {sample_ephem_time.coordinates.time.to_iso8601()[0]}")
    print(f"RA: {pc.min(sample_ephem_time.coordinates.lon)} - {pc.max(sample_ephem_time.coordinates.lon)}")
    print(f"DEC: {pc.min(sample_ephem_time.coordinates.lat)} - {pc.max(sample_ephem_time.coordinates.lat)}")



Object ID: C11QM25

Time: 2025-02-23T00:00:00.000
RA: 131.8765804600289 - 312.20777783150254
DEC: -14.388007767026275 - 51.40213360541426
Object ID: C11QM25

Time: 2025-02-23T00:05:00.000
RA: 131.87519081007588 - 312.2100283360548
DEC: -14.395731775311084 - 50.316578057588345
Object ID: C11QM25

Time: 2025-02-23T00:10:00.000
RA: 131.87379975693307 - 312.2123773385522
DEC: -14.403355028559567 - 50.756042539885584
